In [2]:
import pandas as pd
import numpy as np

import requests
from geopy.geocoders import Nominatim

### Read the Finalized Data

In [3]:
# Read the 'df_GmLgStBpFinal' data
df_GmLgStBpFinal = pd.read_csv("https://media.githubusercontent.com/media/sharmajee499/MinneMUDAC2023/main/Data/df_GmLgStBpFinal.csv")

In [4]:
# Instantiate the Geolocator class(The parameter is just dummy name)
geolocator = Nominatim(user_agent="ballParkLocation")

def fetch_weather(df, startYear: int, endYear:int):

    temp_dict = {'Date': [],
             'parkID': [],
             'City':[],
             'temp_unit': [],
             'time_zone':[],
             'rain_sum':[],
             'snowfall_sum': [],
             'precipitation_sum':[],
             'temp_max': [],
             'temp_min': [],
             }

    # Subset the data
    df = df_GmLgStBpFinal[(df_GmLgStBpFinal['Year'] >= startYear) & (df_GmLgStBpFinal['Year'] <= startYear)]

    for i in range(len(df)):

        try:
            location = geolocator.geocode(df['CITY_StdCap'][i])     

            lati = location.latitude
            longi = location.longitude

            start_date = df['Date_Weather'][i]
            end_date = df['Date_Weather'][i]

            temp_unit = 'fahrenheit'

            # URL to make call
            url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lati}&longitude={longi}&start_date={start_date}&end_date={end_date}&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum&timezone=America%2FChicago&temperature_unit={temp_unit}"
    
            # Call the URL
            response = requests.get(url)

                # Assign the data to the dictionary and it's respective keys
            temp_dict['Date'].append(start_date)
            temp_dict['parkID'].append(df['BallParkID'][i])
            temp_dict['City'].append(df['CITY_StdCap'][i])
            temp_dict['temp_unit'].append(temp_unit)
            temp_dict['time_zone'].append(response.json()['timezone'])
            temp_dict['rain_sum'].append(response.json()['daily']['rain_sum'][0])
            temp_dict['snowfall_sum'].append(response.json()['daily']['snowfall_sum'][0])
            temp_dict['precipitation_sum'].append(response.json()['daily']['precipitation_sum'][0])
            temp_dict['temp_max'].append(response.json()['daily']['temperature_2m_max'][0])
            temp_dict['temp_min'].append(response.json()['daily']['temperature_2m_min'][0])
            print(response.json()['daily']['temperature_2m_min'][0])

        except:
            temp_dict['Date'].append(df['Date'][i])
            temp_dict['parkID'].append(df['BallParkID'][i])
            temp_dict['City'].append(df['CITY_StdCap'][i])
            temp_dict['temp_unit'].append(temp_unit)
            temp_dict['time_zone'].append(None)
            temp_dict['rain_sum'].append(None)
            temp_dict['snowfall_sum'].append(None)
            temp_dict['precipitation_sum'].append(None)
            temp_dict['temp_max'].append(None)
            temp_dict['temp_min'].append(None)

    df_temp = pd.DataFrame(temp_dict)

    file_name = f"{startYear}_{endYear}_weatherData.csv"

    df_temp.to_csv(file_name, index=False)

In [ ]:
fetch_weather(df_GmLgStBpFinal, 2000, 2000)